In [1]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os
import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

import ccxt
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
client = MongoClient()
db=client.fsmbot
trades = {}

context = zmq.asyncio.Context()


In [3]:
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})

In [6]:
# t = db.trades.find_one({'_id': ObjectId("615cc81fa0d373f6b44b88b0")})
# t['curStopResp']

In [4]:
# exchange.create_order('ALGO/USDT', 'market', 'buy', 15)
# os.getenv('TEST')

In [9]:
zContext = zmq.asyncio.Context()

class TradeModel(object):
    def __init__(self, trade): 
        self.trade = trade
        self._id = trade['_id']
        self.symbol = trade['symbol']
        
    def reload_from_db(self, event):
        self.trade = db.trades.find_one({'_id': self._id})
        print('reloaded', self.trade)
        self.symbol = trade['symbol']

    def is_buy_signal(self, event):
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) <= float(self.trade['trigger']):
            # logging.info(f'+BUY triggered for {self.symbol} / {self.trade["_id"]}')
            return True
        # logging.info(f'-BUY triggered for {self.symbol} / {self.trade["_id"]}')
        return False

    def is_sell_signal(self, event):
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) >= float(self.trade['target']):
            # logging.info(f'+SELL triggered for {self.symbol} / {self.trade["_id"]}')
            return True
        # logging.info(f'-SELL triggered for {self.symbol} / {self.trade["_id"]}')
        return False

    def buy_on_exchange(self, event):
        resp = exchange.create_order(self.symbol, 'market', 'buy', float(self.trade["qty"]));
        db.trades.update_one({'_id' : self.trade['_id'] }, {
        '$set': {
            'buyResp': resp
          }})

    def sell_on_exchange(self, event):
        resp = exchange.create_order(self.symbol, 'market', 'sell', float(self.trade["qty"]));
        db.trades.update_one({'_id' : self.trade['_id'] }, {
        '$set': {
            'sellResp': resp
          }})

    def set_stop_on_exchange(self, event):
        resp = exchange.create_order(self.symbol, 'STOP_LOSS_LIMIT', 'sell', float(self.trade["qty"]), 
            0.90 * float(self.trade["stop"]), {'stopPrice': float(self.trade["stop"]),'type': 'stopLimit'})
        db.trades.update_one({'_id' : self.trade['_id'] }, {
        '$set': {
            'curStopResp': resp
          }})
        # self.trade['curStopResp'] = resp

    def remove_stop_on_exchange(self, event):
        resp = exchange.cancel_order(self.trade['curStopResp']['id'], symbol=self.symbol)
      
    
    def sellTriggered(self, event):
        True

    def machine_state_change(self, event):
        db.trades.update_one({'_id' : self.trade['_id'] }, {
        '$set': {
            'state': self.state
          }})

states = [
    'ERROR',
    'DISABLED',
    'LIVE',
    'LONG',
    'STOPPED',
    'SOLD'
]
transitions = [
    { 'trigger': 'start', 'source': 'DISABLED', 'dest': 'LIVE'},
    { 'trigger': 'disable', 'source': 'LIVE', 'dest': 'DISABLED'},
    { 'trigger': 'tick', 'source': 'LIVE', 'dest': 'LONG', 'conditions': 'is_buy_signal', 'before' : ['buy_on_exchange', 'set_stop_on_exchange', 'reload_from_db']},
    { 'trigger': 'movestop', 'source': 'LONG', 'dest': 'LONG' ,'before': ['remove_stop_on_exchange', 'set_stop_on_exchange']},
    { 'trigger': 'stopped', 'source': 'LONG', 'dest': 'STOPPED'},
    { 'trigger': 'tick', 'source': 'LONG', 'dest': 'SOLD' ,'conditions': 'is_sell_signal', 'before': ['remove_stop_on_exchange', 'sell_on_exchange', 'reload_from_db']},
]


In [6]:
# loop = asyncio.get_running_loop()

In [10]:
# list(dbTrade)
for trade in db.trades.find():
    if not trade['symbol'] in trades: trades[trade['symbol']] = []
    tradeModel = TradeModel(trade)
    tradeMachine = Machine(tradeModel, states=states, transitions=transitions, send_event=True, initial='DISABLED', 
        after_state_change='machine_state_change')
    tradeMachine.set_state(trade['state'])
    trades[trade['symbol']].append({'machine': tradeMachine, 'model': tradeModel})

# trades
#     pass
# dbTrades = db.trades
# list(db.trades.find())




In [8]:
# [o['id'] for o in exchange.fetch_open_orders('ALGOUSDT')]
# exchange.cancel_order('683898256', symbol='ALGOUSDT')

In [9]:


# Process 5 updates
async def run_ticker():
    socket = context.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        # print(symbol)
        if symbol in trades:        
            for trade in trades[symbol]:
                model = trade['model']
                # [tradeMachine, tradeModel] = trade
                # print(trade['model'])
                if model.state in ['LIVE', 'LONG']:
                    # print(ticker)
                    # print(symbol)
                    model.tick(ticker=ticker)
        # pprint(ticker)
        # await asyncio.sleep(0.01)
    
await run_ticker()
    # print('received', ticker)
    # zipcode, temperature, relhumidity = string.split()
    # total_temp += int(temperature)
# loop = asyncio.get_event_loop()
# loop.create_task(run_ticker())


INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Executed callback 'reload_from_db'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_change'


reloaded {'_id': ObjectId('615cc81fa0d373f6b44b88b0'), 'symbol': 'ALGOUSDT', 'trigger': '2', 'stop': '1.5', 'target': '3', 'state': 'LIVE', 'exchange': 'binance', 'qty': '15', 'buyResp': {'info': {'symbol': 'ALGOUSDT', 'orderId': '683918052', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S821848d59428d87663b00555', 'transactTime': '1633500134900', 'price': '0.00000000', 'origQty': '15.00000000', 'executedQty': '15.00000000', 'cummulativeQuoteQty': '28.16700000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '1.87780000', 'qty': '15.00000000', 'commission': '0.01500000', 'commissionAsset': 'ALGO', 'tradeId': '65961683'}]}, 'id': '683918052', 'clientOrderId': 'x-R4BD3S821848d59428d87663b00555', 'timestamp': 1633500134900, 'datetime': '2021-10-06T06:02:14.900Z', 'lastTradeTimestamp': None, 'symbol': 'ALGO/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 1.8778000000000001, 'stopPrice': None, 'amount'

KeyError: 'curStopResp'

In [12]:
# trades['SOLUSDT'][0]['model'].disable()
model = trades['ALGOUSDT'][0]['model']
# trades['ALGOUSDT'][0]['model'].tick(ticker={'curDayClose': 1})
# loop = asyncio.get_event_loop()
# loop.create_task(model.tick(ticker={'curDayClose': 1}))
# asyncio.gather([model.tick(ticker={'curDayClose': 1})])
# asyncio.get_event_loop().run_until_complete()

# asyncio.run(asyncio.wait([model.tick(ticker={'curDayClose': 1})]))
# await asyncio.wait(model.tick(ticker={'curDayClose': 1}))
# model.tick(ticker={'curDayClose': "1"})
# loop = asyncio.get_event_loop()
# loop.run_until_complete()
# model.start()
# model.disable()
# trades['ALGOUSDT'][0]['model'].state
# trades['ALGOUSDT'][0]['model'].trade['curStopResp']['id']
trade = trades['ALGOUSDT'][0]['model'].trade
# trade.tick(ticker = {'curStopResp': })
# .start()

'683918059'

In [ ]:
float("")